In [ ]:
!pip install torch-summary

In [ ]:
from google.colab import files
files.upload()

{}

In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets list

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
!kaggle datasets download -d warcoder/tyre-quality-classification

In [ ]:
import zipfile

In [ ]:
dataset_zip =  zipfile.ZipFile('tyre-quality-classification.zip', 'r')

dataset_zip.extractall()
dataset_zip.close()

In [ ]:
Id=[]
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content/Digital images of defective and good condition tyres'):
    for filename in filenames:
        Id.append(os.path.join(dirname, filename))

In [ ]:
train=pd.DataFrame()
train=train.assign(filename=Id)
train.head()

In [ ]:
train['label']=train['filename']
train['label']=train['label'].str.replace('/content/Digital images of defective and good condition tyres/','')
train.head()

In [ ]:
train['label'] = train['label'].str.split('/').str[0]
train.head()

In [ ]:
count=(train.label.value_counts()).to_frame()
count.reset_index(inplace=True)
count.columns=['label','count']
count

In [ ]:
from sklearn.model_selection import train_test_split
train,test= train_test_split( train, test_size=0.2, random_state=42,shuffle=True, stratify=train['label'])

In [ ]:
count=(train.label.value_counts()).to_frame()
count.reset_index(inplace=True)
count.columns=['label','count']
count

In [ ]:
train=train.sample(n=len(train))
train.reset_index(inplace=True,drop=True)
train.head()

In [ ]:
test=test.sample(n=len(test))
test.reset_index(inplace=True,drop=True)
test.head()

In [ ]:
import os
import cv2
import multiprocessing
from functools import partial
import pandas as pd
import uuid

def _process_df_chunk(chunk, file_col, label_col, dest_folder):
    def _process_image(file_path, label):
        try:
            # Load the image
            img = cv2.imread(file_path)
            # Check if the image size is empty
            if img is None or img.size == 0:
                print(f"Error: Could not read file {file_path}")
                return
            # Resize to 512x512
            img = cv2.resize(img, (512, 512))
            # Save the image to the folder with label name
            folder_path = os.path.join(dest_folder, str(label))

            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
            file_name = os.path.basename(file_path)
            # Append a unique identifier to the file name to avoid overwriting
            name, extension = os.path.splitext(file_name)
            save_path = os.path.join(folder_path, f"{name}_{uuid.uuid4().hex}{extension}")
            cv2.imwrite(save_path, img)
        except Exception as e:
            print(f"Error: {e}")

    for _, row in chunk.iterrows():
        _process_image(row[file_col], row[label_col])

def resize_save_image(df, file_col, label_col, dest_folder):
    try:
        # Determine the number of processes to use
        num_processes = multiprocessing.cpu_count()

        df_chunks = [df[i:i + num_processes] for i in range(0, len(df), num_processes)]

        # Create a partial function with fixed arguments
        func = partial(_process_df_chunk, file_col=file_col, label_col=label_col, dest_folder=dest_folder)

        # Run the function in parallel on the DataFrame chunks
        with multiprocessing.Pool(processes=num_processes) as pool:
            pool.map(func, df_chunks)
    except Exception as e:
        print(f"Error: {e}")

In [ ]:
newpath = r'./tyre-quality-classification/train'
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [ ]:
resize_save_image(train, file_col='filename', label_col='label' ,dest_folder='./tyre-quality-classification/train')

In [ ]:
newpath = r'./tyre-quality-classification/test'
if not os.path.exists(newpath):
    os.makedirs(newpath)

In [ ]:
resize_save_image(test, file_col='filename', label_col='label' ,dest_folder='./tyre-quality-classification/test')

In [ ]:
train.shape

In [ ]:
train

In [ ]:
test.shape

In [ ]:
test

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from PIL import Image
from IPython.display import display
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.optim.lr_scheduler import StepLR
from torchsummary import summary
from tqdm import tqdm
import torchvision.models as models
import PIL
from torchvision.models import resnet50, ResNet50_Weights
from torch.utils.data import Dataset
import glob
from pathlib import Path
from imblearn.over_sampling import RandomOverSampler
import gc
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, f1_score, classification_report
from numpy.random import seed

In [ ]:
seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=20),
    transforms.GaussianBlur(5, sigma=0.60),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomErasing()
])

test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
train_dir = '/content/tyre-quality-classification/train'
test_dir = '/content/tyre-quality-classification/test'

train_data = datasets.ImageFolder(train_dir, transform=train_transform)
test_data = datasets.ImageFolder(test_dir, transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=32, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_data, shuffle=False, batch_size=32, pin_memory=True)

In [ ]:
import torchvision
def show_transformed_img(dataset):
  loader = torch.utils.data.DataLoader(dataset, batch_size= 6, shuffle=True)
  batch = next(iter(loader))
  images, labels = batch

  grid = torchvision.utils.make_grid(images, nrow = 3)
  plt.figure(figsize=(11,11))
  plt.imshow(np.transpose(grid, (1,2,0)))
  print('labels', labels)

In [ ]:
show_transformed_img(test_data)

In [ ]:
def set_device():
  if torch.cuda.is_available():
    dev = 'cuda:0'
  else:
    dev = 'cpu'

  return torch.device(dev)

In [ ]:
def train_nn(model, train_loader, test_loader, criterion, optimizer, n_epochs ):
  device = set_device()
  best_acc = 0

  for epoch in range(n_epochs):
    print("Epoch number %d" % (epoch + 1))
    model.train
    running_loss = 0.0
    running_correct = 0.0
    total = 0

    for data in train_loader:
      images, labels = data
      images = images.to(device)
      labels = labels.to(device)
      total += labels.size(0)

      optimizer.zero_grad()

      outputs = model(images)

      _, predicted = torch.max(outputs.data, 1)

      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item()
      running_correct += (labels == predicted).sum().item()

    epoch_loss = running_loss/len(train_loader)
    epoch_acc = 100.0 * running_correct / total

    print("     - Training dataset. Got %d out of %d images correctly (%.3f%%). Epoch loss: %.3f" % (running_correct, total, epoch_acc, epoch_loss))

    test_dataset_acc = evaluate_model_on_test_set(model, test_loader)

    if(test_dataset_acc > best_acc):
      best_acc = test_dataset_acc
      save_checkpoint(model, epoch, optimizer, best_acc)

  print("Finished")
  return model

In [ ]:
def evaluate_model_on_test_set(model, test_loader):
  model.eval()
  predicted_correctly_on_epoch = 0
  total = 0
  device = set_device()

  with torch.no_grad():
    for data in test_loader:
      images, labels = data
      images = images.to(device)
      labels = labels.to(device)
      total += labels.size(0)

      outputs = model(images)

      _, predicted = torch.max(outputs.data, 1)
      predicted_correctly_on_epoch += (predicted ==labels).sum().item()

  epoch_acc = 100.0 * predicted_correctly_on_epoch / total
  print("     - Testing dataset. Got %d out of %d images correctly (%.3f%%)" % (predicted_correctly_on_epoch, total, epoch_acc))

  return epoch_acc

In [ ]:
def save_checkpoint(model, epoch, optimizer, best_acc):
  state = {
      'model': model.state_dict(),
      'epoch': epoch + 1,
      'optimizer': optimizer.state_dict(),
      'best_accuracy': best_acc,
  }
  torch.save(state, 'rubber_guardian_checkpoint.pth.tar')

In [ ]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

resnet50_model = models.resnet50(pretrained = True)
num_classes = 2

def set_parameter_requires_grad(model, feature_extracting=True):
    if feature_extracting:
        for name, param in model.named_parameters():
            param.requires_grad = False

resnet50_model.fc = nn.Sequential(
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Linear(256, num_classes)
)

device = set_device()
resnet_50_model = resnet50_model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9, weight_decay=0.003)



In [ ]:
train_nn(resnet50_model, train_loader, test_loader, loss_fn, optimizer, 150)

In [ ]:
checkpoint = torch.load('/content/rubber_guardian_checkpoint.pth.tar')

In [ ]:
print(checkpoint['epoch'])
print(checkpoint['best_accuracy'])

In [ ]:
resnet50_model = models.resnet50()
num_classes = 2
resnet50_model.fc = nn.Sequential(
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 256),
    nn.ReLU(),
    nn.Linear(256, num_classes)
)
resnet50_model.load_state_dict(checkpoint['model'])

In [ ]:
torch.save(resnet50_model, 'best_rgmodel2.pth')

In [ ]:
files.download('/content/best_rgmodel2.pth')

In [ ]:
import torchvision
import torch
import PIL.Image as Image

In [ ]:
classes = [
    "defective",
    'good'
]

In [ ]:
model = torch.load('/content/best_rgmodel.pth')

In [ ]:
def classify(model, image_transforms, image_path, classes):
  model = model.eval()
  image = Image.open(image_path)
  image = image_transforms(image).float()
  image = image.unsqueeze(0)

  output = model(image)
  _, predicted = torch.max(output.data, 1)

  print(classes[predicted.item()])

In [ ]:
#classify(model, test_transform, "/content/Defective (7).jpg", classes)

In [ ]:
#classify(model, test_transform, "/content/Defective (63).jpg", classes)

In [ ]:
#classify(model, test_transform, "/content/Defective (45).jpg", classes)

In [ ]:
#classify(model, test_transform, "/content/Defective (2).jpg", classes)

In [ ]:
#classify(model, test_transform, "/content/Defective (11).jpg", classes)

In [ ]:
#classify(model, test_transform, "/content/good (1).jpg", classes)

In [ ]:
#classify(model, test_transform, "/content/good (542).jpg", classes)

In [ ]:
#classify(model, test_transform, "/content/good (63).jpg", classes)

In [ ]:
#classify(model, test_transform, "/content/good (682).jpg", classes)

In [ ]:
#classify(model, test_transform, "/content/good (9).jpg", classes)

In [ ]:
#classify(model, test_transform, "/content/google_tyre_defect(1).jpg", classes)

In [ ]:
#classify(model, test_transform, "/content/google_tyre_good(1).jpg", classes)